# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/johnrick/Desktop/udacity/cassandra_project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
df = pd.read_csv('event_datafile_new.csv')

In [6]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
# create table/ database?
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




artist
firstName of user
gender of user
item number in session
last name of user
length of the song
level (paid or free song)
location of the user
sessionId
song title
userId

In [10]:
# Cassandra data types
# https://www.tutorialspoint.com/cassandra/cassandra_cql_datatypes.htm

In [11]:
#Create Table for each querie
# artist, song, song length, sessionID, itemInSession
session.execute('''CREATE TABLE IF NOT EXISTS music_history (sessionId INT, itemNoInSession INT, artist TEXT, song_title TEXT, 
song_length float, PRIMARY KEY(sessionId, itemNoInSession));''')

In [12]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

In [13]:
cols = df.columns.tolist()

In [14]:
nums = [i for i in range(len(df.columns))]

In [15]:
col_lines = dict(zip(cols, nums))

In [16]:
col_lines

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [17]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO music_history (sessionId, 
        itemNoInSession, 
        artist, 
        song_title, 
        song_length) 
        VALUES (%s, %s, %s, %s, %s)"""
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT artist, song_title, song_length FROM music_history WHERE sessionId = 338 AND itemNoInSession =4;")

In [20]:
# # view 1
# for row in rows:
#     print(row)

In [21]:
# # view 2
# for row in rows:
#     print(row.artist, row.song_title, row.song_length)

In [22]:
# view 3
pd.DataFrame(rows)

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [23]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
                    

In [24]:
# reminder
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26


In [25]:
# reminder
col_lines

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [26]:
# Create table
session.execute("""CREATE TABLE IF NOT EXISTS users (userId INT, sessionId INT, itemInSession INT,
artist TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY (userId, sessionId, itemInSession));""")

In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO users (userId, sessionId, itemInSession, artist, song, firstName, lastName) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [28]:
## query2: name of artist, song (sorted by itemInSession) and user (first and last name) 
## for userid = 10, sessionid = 182
rows = session.execute("SELECT artist, song, firstName, lastName FROM users WHERE userId = 10 AND sessionId = 182;")

In [29]:
pd.DataFrame(rows)

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [30]:
# #double check to see if everything is in order
# rows = session.execute("SELECT * FROM users WHERE userId = 10 AND sessionId = 182;")

In [31]:
# #yes it is
# pd.DataFrame(rows)

In [32]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                    

In [33]:
# session.execute("DROP TABLE IF EXISTS user_history;")

In [34]:
# user_history: userId, first, last, song
session.execute('''CREATE TABLE IF NOT EXISTS user_history (song TEXT, userId INT, firstName TEXT, lastName TEXT, PRIMARY KEY(song, userId));''')

In [35]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO user_history (song, userId, firstName, lastName) 
        VALUES (%s, %s, %s, %s)"""
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [36]:
# every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                    
rows = session.execute("SELECT firstName, lastName FROM user_history WHERE song = 'All Hands Against His Own'")

In [37]:
pd.DataFrame(rows)

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [38]:
## TO-DO: Drop the table before closing out the sessions
drop_table1 = "DROP TABLE IF EXISTS music_history"
drop_table2 = "DROP TABLE IF EXISTS users"
drop_table3 = "DROP TABLE IF EXISTS user_history"
drop_tables = [drop_table1, drop_table2, drop_table3]

for i in drop_tables:
    try: 
        session.execute(i)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()